3. Простой пример: «конструктор» HTML‑документа

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import List

# ---------- Product ----------
class HTMLDocument:
    def __init__(self) -> None:
        self._parts: List[str] = []

    def add(self, html: str) -> None:
        self._parts.append(html)

    def render(self) -> str:
        return "\n".join(self._parts)

# ---------- Builder ----------
class HTMLBuilder(ABC):
    @abstractmethod
    def add_title(self, text: str) -> "HTMLBuilder": ...
    @abstractmethod
    def add_paragraph(self, text: str) -> "HTMLBuilder": ...
    @abstractmethod
    def add_list(self, items: List[str]) -> "HTMLBuilder": ...
    @abstractmethod
    def build(self) -> HTMLDocument: ...

# ---------- Concrete Builder (fluent API) ----------
class SimpleHTMLBuilder(HTMLBuilder):
    def __init__(self) -> None:
        self._doc = HTMLDocument()

    def add_title(self, text: str) -> "SimpleHTMLBuilder":
        self._doc.add(f"<h1>{text}</h1>")
        return self

    def add_paragraph(self, text: str) -> "SimpleHTMLBuilder":
        self._doc.add(f"<p>{text}</p>")
        return self

    def add_list(self, items: List[str]) -> "SimpleHTMLBuilder":
        lis = "".join(f"<li>{i}</li>" for i in items)
        self._doc.add(f"<ul>{lis}</ul>")
        return self

    def build(self) -> HTMLDocument:
        return self._doc

# ---------- Клиентский код ----------
if __name__ == "__main__":
    html = (
        SimpleHTMLBuilder()
        .add_title("Привет, мир!")
        .add_paragraph("Это документ, собранный строителем.")
        .add_list(["пункт 1", "пункт 2", "пункт 3"])
        .build()
    )
    print(html.render())


<h1>Привет, мир!</h1>
<p>Это документ, собранный строителем.</p>
<ul><li>пункт 1</li><li>пункт 2</li><li>пункт 3</li></ul>


4. Сложный пример: SQL‑конструктор запросов
Задача: безопасно и удобно собирать сложные SQL‑запросы (SELECT‑ы с JOIN‑ами, WHERE, ORDER).
Хотим:

«чейниться» (.select().where().order_by()),

подставлять параметры через плейсхолдеры (избежать SQL‑инъекций),

поддерживать разные диалекты (PostgreSQL, SQLite).

In [2]:
"""
Упрощённый, но реалистичный Query Builder.
Поддерживает PostgreSQL ($1, $2, …) и SQLite (?) плейсхолдеры.
"""
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import List, Tuple, Sequence

# ---------- Product ----------
class Query:
    def __init__(self, sql: str, params: Sequence[object]) -> None:
        self.sql = sql
        self.params = params

    def __str__(self) -> str:
        return f"{self.sql}  -- params={self.params}"

# ---------- Builder ----------
class QueryBuilder(ABC):
    def __init__(self) -> None:
        self._select: List[str] = []
        self._table: str = ""
        self._joins: List[str] = []
        self._where: List[str] = []
        self._order: List[str] = []
        self._limit: int | None = None
        self._params: List[object] = []

    # --- fluent шаги ---
    def select(self, *fields: str) -> "QueryBuilder":
        self._select.extend(fields or ["*"])
        return self

    def from_(self, table: str) -> "QueryBuilder":
        self._table = table
        return self

    def join(self, table: str, on: str) -> "QueryBuilder":
        self._joins.append(f"JOIN {table} ON {on}")
        return self

    def where(self, expr: str, *params: object) -> "QueryBuilder":
        placeholder_expr = self._convert_placeholders(expr, len(params))
        self._where.append(placeholder_expr)
        self._params.extend(params)
        return self

    def order_by(self, *fields: str) -> "QueryBuilder":
        self._order.extend(fields)
        return self

    def limit(self, n: int) -> "QueryBuilder":
        self._limit = n
        return self

    # --- итог ---
    @abstractmethod
    def build(self) -> Query: ...

    # --- protected helper ---
    @abstractmethod
    def _convert_placeholders(self, expr: str, n: int) -> str: ...

# ---------- Concrete Builders ----------
class PgQueryBuilder(QueryBuilder):
    def _convert_placeholders(self, expr: str, n: int) -> str:
        """Заменяем '?' на $1, $2 …"""
        for i in range(1, n + 1):
            expr = expr.replace("?", f"${i}", 1)
        return expr

    def build(self) -> Query:
        if not self._table:
            raise ValueError("FROM is mandatory")
        parts = [
            f"SELECT {', '.join(self._select) or '*'}",
            f"FROM {self._table}",
            *self._joins,
        ]
        if self._where:
            parts.append("WHERE " + " AND ".join(self._where))
        if self._order:
            parts.append("ORDER BY " + ", ".join(self._order))
        if self._limit is not None:
            parts.append(f"LIMIT {self._limit}")
        sql = " ".join(parts)
        return Query(sql, tuple(self._params))

class SqliteQueryBuilder(QueryBuilder):
    def _convert_placeholders(self, expr: str, n: int) -> str:
        # SQLite уже использует '?', поэтому ничего не меняем
        return expr

    def build(self) -> Query:
        # Реализация идентична, можно вынести в миксин, но оставим явно
        if not self._table:
            raise ValueError("FROM is mandatory")
        parts = [
            f"SELECT {', '.join(self._select) or '*'}",
            f"FROM {self._table}",
            *self._joins,
        ]
        if self._where:
            parts.append("WHERE " + " AND ".join(self._where))
        if self._order:
            parts.append("ORDER BY " + ", ".join(self._order))
        if self._limit is not None:
            parts.append(f"LIMIT {self._limit}")
        sql = " ".join(parts)
        return Query(sql, tuple(self._params))

# ---------- Director (рецепты) ----------
class ReportQueries:
    """Готовые сценарии: Director вызывает шаги билдера в нужном порядке."""
    def __init__(self, builder: QueryBuilder) -> None:
        self._b = builder

    def top_customers(self, year: int, limit: int = 10) -> Query:
        return (
            self._b
            .select("c.name", "SUM(o.total) AS revenue")
            .from_("customers c")
            .join("orders o", "o.customer_id = c.id")
            .where("o.status = ?", "PAID")
            .where("EXTRACT(year FROM o.date) = ?", year)
            .group_by("c.name")
            .order_by("revenue DESC")
            .limit(limit)
            .build()
        )

# Добавим недостающий метод group_by «на лету» (трюк monkey‑patch для краткости)
def _group_by(self: QueryBuilder, *fields: str) -> "QueryBuilder":
    self._group = ", ".join(fields)
    return self
QueryBuilder.group_by = _group_by  # type: ignore

# ---------- Демонстрация ----------
if __name__ == "__main__":
    # PostgreSQL
    pg_query = (
        PgQueryBuilder()
        .select("*")
        .from_("users")
        .where("age > ?", 18)
        .order_by("created_at DESC")
        .limit(5)
        .build()
    )
    print("PG:", pg_query)

    # Используем Director
    report = ReportQueries(PgQueryBuilder())
    print("Report:", report.top_customers(2024))

    # SQLite
    sqlite_query = (
        SqliteQueryBuilder()
        .select("id", "email")
        .from_("subscribers")
        .where("confirmed = ?", True)
        .build()
    )
    print("SQLite:", sqlite_query)


PG: SELECT * FROM users WHERE age > $1 ORDER BY created_at DESC LIMIT 5  -- params=(18,)
Report: SELECT c.name, SUM(o.total) AS revenue FROM customers c JOIN orders o ON o.customer_id = c.id WHERE o.status = $1 AND EXTRACT(year FROM o.date) = $1 ORDER BY revenue DESC LIMIT 10  -- params=('PAID', 2024)
SQLite: SELECT id, email FROM subscribers WHERE confirmed = ?  -- params=(True,)
